In [1]:
import pyautogui
import time

In [8]:
i = 0

while 1:
    time.sleep(1)
    screenshot = pyautogui.screenshot()
    screenshot.save(f"data/fight/{i}.png")
    i += 1

KeyboardInterrupt: 

In [11]:
l = 20
h = 14